### Importing required libraries modules and functions.

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets, transforms, utils
from torch.utils.data.sampler import SubsetRandomSampler
from torch.nn import functional as F
import matplotlib.pyplot as plt
import random

### Loading MNIST Dataset

We are doing binary classification, so we need only 1 and 0 digits.

In [2]:
import warnings
warnings.filterwarnings('ignore')

batch_size = 64

train_data = datasets.MNIST('./data/anupam-data/pytorch/data/', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
test_data = datasets.MNIST('./data/anupam-data/pytorch/data/', train=False, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

subset_indices = ((train_data.train_labels == 0) + (train_data.train_labels == 1)).nonzero().view(-1)

train_loader = torch.utils.data.DataLoader(dataset=train_data, 
                                           batch_size=batch_size, 
                                           shuffle=False,
                                           sampler=SubsetRandomSampler(subset_indices)
                                          )

subset_indices = ((test_data.test_labels == 0) + (test_data.test_labels == 1)).nonzero().view(-1)

test_loader = torch.utils.data.DataLoader(dataset=test_data, 
                                          batch_size=batch_size,
                                          shuffle=False,
                                          sampler=SubsetRandomSampler(subset_indices)
                                         )


In [3]:
import warnings
warnings.filterwarnings('ignore')
torch.manual_seed(42)

batch_size = 64
input_size = 784  #(dimension of image 28 * 28)
num_classes = 1   #(just -1 and 1 image)
learning_rate = 0.0001 ## step size used by SGD 
momentum = 0.015 ## Momentum is a moving average of our gradients (helps to keep direction)

class Regress_Loss(nn.modules.Module):    
    def __init__(self):
        super(Regress_Loss,self).__init__()
    def forward(self, outputs, labels):
        batch_size = outputs.size()[0]
        return torch.sum(torch.log(1 + torch.exp(-(outputs.t()*labels))))/batch_size

class SVM_Loss(nn.modules.Module):    
    def __init__(self):
        super(SVM_Loss,self).__init__()
    def forward(self, outputs, labels):
         return torch.sum(torch.clamp(1 - outputs.t()*labels, min=0))/batch_size
        

#Logistic regression model and Loss

models = [Regress_Loss(), SVM_Loss()]



for j in range(8):

    logistics_model = nn.Linear(input_size, num_classes)
    
    optimizers = [torch.optim.SGD(logistics_model.parameters(), lr=learning_rate, weight_decay = 0),
                  torch.optim.SGD(logistics_model.parameters(), lr=learning_rate, weight_decay = 1),
                  torch.optim.Adam(logistics_model.parameters(), lr=learning_rate, weight_decay = 0),
                  torch.optim.Adam(logistics_model.parameters(), lr=learning_rate, weight_decay = 1),
                  torch.optim.SGD(logistics_model.parameters(), lr=learning_rate, weight_decay = 0),
                  torch.optim.SGD(logistics_model.parameters(), lr=learning_rate, weight_decay = 1),
                  torch.optim.Adam(logistics_model.parameters(), lr=learning_rate, weight_decay = 0),
                  torch.optim.Adam(logistics_model.parameters(), lr=learning_rate, weight_decay = 1)]
    
    if j <= 3:
        loss_criteria = models[0]

    else:
        loss_criteria = models[1]
    
    optimizer = optimizers[j]
    total_step = len(train_loader)
    
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, 28*28)            
        labels = Variable(2*(labels.float()-0.5))

        outputs = logistics_model(images)               
        loss = loss_criteria(outputs, labels)    

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()   

    # Test the Model
    correct = 0.
    total = 0.
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)

        outputs_test = torch.sigmoid(logistics_model(images))
        
        predicted = outputs_test.data >= 0.5 

        total += labels.size(0) 

        correct += (predicted.view(-1).long() == labels).sum()
      
    if j == 0:
        print('Logistic Regression Accuracy with SGD optimizer without regularization: %f %%' % (100 * (correct.float() / total)))
    elif j == 1:
        print('Logistic Regression Accuracy with SGD optimizer with regularization: %f %%' % (100 * (correct.float() / total)))
    elif j == 2:
        print('Logistic Regression Accuracy with Adam optimizer without regularization: %f %%' % (100 * (correct.float() / total)))
    elif j == 3:
        print('Logistic Regression Accuracy with Adam optimizer with regularization: %f %%' % (100 * (correct.float() / total)))
    elif j == 4:
        print('Support Vector Machine Accuracy with SGD optimizer without regularization: %f %%' % (100 * (correct.float() / total)))
    elif j == 5:
        print('Support Vector Machine Accuracy with SGD optimizer with regularization: %f %%' % (100 * (correct.float() / total)))
    elif j == 6:
        print('Support Vector Machine Accuracy with Adam optimizer without regularization: %f %%' % (100 * (correct.float() / total)))
    else:
        print('Support Vector Machine Accuracy with Adam optimizer with regularization: %f %%' % (100 * (correct.float() / total)))
    
    logistics_model.reset_parameters()  
    


Logistic Regression Accuracy with SGD optimizer without regularization: 97.163124 %
Logistic Regression Accuracy with SGD optimizer with regularization: 99.101654 %
Logistic Regression Accuracy with Adam optimizer without regularization: 99.243500 %
Logistic Regression Accuracy with Adam optimizer with regularization: 99.385345 %
Support Vector Machine Accuracy with SGD optimizer without regularization: 99.338058 %
Support Vector Machine Accuracy with SGD optimizer with regularization: 99.432625 %
Support Vector Machine Accuracy with Adam optimizer without regularization: 99.290779 %
Support Vector Machine Accuracy with Adam optimizer with regularization: 99.763596 %
